In [1]:
import numpy as np
import pandas as pd 


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
clf = RandomForestClassifier(n_estimators = 100) 

from sklearn import metrics

def load (input):
    
    data = pd.read_csv(input, delimiter = ",")
    extraced_ssid_level_pivot =data.pivot(columns=['bssid'], values='level')
    
    output =pd.concat([data, extraced_ssid_level_pivot] , axis=1)
    if 'level' in output:
        del output['level']
    if 'ssid' in output:
        del output['ssid']
    if 'bssid' in output:
        del output['bssid']
    return output

def get_features_and_target(df , target_to_predicte):
    target=df[target_to_predicte]
    if target_to_predicte in df:
        del df[target_to_predicte]
    features=df
    return features, target

dataset  = load('donnees.txt') ;   # on souhaitera ajouter des donnees dans ce dataset
#dataset['salleid'] = dataset['salleid'].astype(str)

In [2]:

dataset = dataset.groupby('positionid').agg("max").fillna(100)
dataset

,batimentid,salleid,floorid,centrefrequence0,frequency,00:4e:35:c8:ce:24,1c:28:af:b2:95:a0,1c:28:af:b2:95:a1,1c:28:af:b2:95:a2,1c:28:af:b2:95:a3,...,9c:1c:12:1d:9e:40,9c:1c:12:1d:9e:41,9c:1c:12:1d:9e:42,9c:1c:12:1d:9e:43,9c:1c:12:1d:9e:45,bc:9f:e4:8b:52:a0,bc:9f:e4:8b:52:a2,bc:9f:e4:8b:52:a3,bc:9f:e4:8b:52:a4,bc:9f:e4:8b:52:a5
positionid,,,,,,,,,,,,,,,,,,,,,
1,1,113,1,5745,5745,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-88.0,-74.0,-73.0,-73.0
2,1,113,1,5785,5785,100.0,100.0,-86.0,-86.0,100.0,...,100.0,100.0,100.0,100.0,100.0,-71.0,-66.0,-69.0,-68.0,-70.0
3,1,113,1,5785,5785,100.0,100.0,-84.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,-85.0,100.0
4,1,113,1,5785,5785,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,-75.0,100.0,100.0
5,1,113,1,5785,5785,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,100.0,100.0,100.0
6,1,113,1,5785,5785,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,100.0,100.0,100.0
7,1,118,1,5785,5785,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,100.0,100.0,100.0
8,1,118,1,5785,5785,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,-88.0,100.0,100.0,100.0,100.0,100.0
9,1,118,1,5785,5785,-90.0,100.0,100.0,100.0,100.0,...,-87.0,-86.0,-87.0,-88.0,100.0,100.0,100.0,100.0,100.0,100.0


In [3]:
from sklearn.neighbors import KNeighborsClassifier
def learn_and_generate_model() : 
    X, y =get_features_and_target(dataset, 'salleid')
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.50, random_state=42)
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X_train, np.asarray(y_train, dtype="|S6"))
    y_pred =  neigh.predict(X_test)
    # metrics are used to find accuracy or error   
    print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(np.asarray(y_test, dtype="|S6"), y_pred))
    from sklearn.metrics import confusion_matrix
    return neigh , confusion_matrix(np.asarray(y_test, dtype="|S6"), y_pred)

learn_and_generate_model()

ACCURACY OF THE MODEL:  0.6923076923076923


array([[0, 1, 0, 2, 0],
       [0, 2, 0, 0, 0],
       [0, 1, 3, 0, 0],
       [0, 0, 0, 2, 0],
       [0, 0, 0, 0, 2]])

In [4]:
import pickle
from sklearn import tree
pickle.dump(clf, open('model.pkl','wb'))

In [11]:
from flask import Flask, request, redirect, url_for, flash, jsonify
import numpy as np
import pickle as p
import json
from pandas import json_normalize

arr =np.array([[1, 2, 3], [4, 5, 6]])
app = Flask(__name__)

#res = request.json()
#df = pd.DataFrame(data)

@app.route('/predict', methods=['GET','POST'])
def makecalc():
    print(request.json)
    majdataset = dataset.append(request.json, ignore_index=True)
    majdataset = majdataset.fillna(100)
    last_scan = majdataset[-1:].values
    salle_predicted = model.predict(last_scan)
    salle_predicted_json = np.array2string((salle_predicted))
    return request.json

@app.route('/app', methods=['GET','POST'])
def makecalcc():
    print(request.json)
    dataset.append(request.json, ignore_index=True)
    print(dataset)
    return request.json
if __name__ == '__main__':
    modelfile = 'model.pkl'
    model = p.load(open(modelfile, 'rb'))
    app.run(debug=False, host='0.0.0.0')
# ici le serveur traite les donnees qu'il recoit 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


In [ ]:
newDfObj = dataset.append({'salleid': '255',
                         'centrefrequence0': '5888'}, ignore_index=True)
newDfObj
newDfObj =newDfObj.fillna(100)
del newDfObj['salleid']  
 
inp = newDfObj[-1:].values
inp

In [49]:
c = {'0': {'ssid': 'UnivToulon', 'bssid': '1c:28:af:ce:e7:01', 'level': -33, 'centerFreq0': 2437, 'frequency': 2437}, '1':
 {'ssid': 'eduroam', 'bssid': '1c:28:af:ce:e7:00', 'level': -33, 'centerFreq0': 2437, 'frequency': 2437}, 
'2': {'ssid': 'visiteurs', 'bssid': '1c:28:af:ce:e7:02', 'level': -33, 'centerFreq0': 2437, 'frequency': 2437}}

In [73]:
c['0']
new = pd.DataFrame.from_dict(c)


d = dict()
for i in range (len(c)):
    print(i)
    print(c['0']['bssid'])
    print(c['0']['level'])
    #d[c['0']['bssid']].append(c['0']['level'])
    d.update( {'Germany' : 49} )
majdataset = dataset.append(request.json, ignore_index=True)
    

0
1c:28:af:ce:e7:01
-33
1
1c:28:af:ce:e7:01
-33
2
1c:28:af:ce:e7:01
-33


RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request.  Consult the documentation on testing for
information about how to avoid this problem.

In [34]:
extraced_ssid_level_pivot =df.pivot(columns=['bssid'], values='level')

KeyError: 'bssid'

In [21]:
dataset

,batimentid,floorid,centrefrequence0,frequency,00:4e:35:c8:ce:24,1c:28:af:b2:95:a0,1c:28:af:b2:95:a1,1c:28:af:b2:95:a2,1c:28:af:b2:95:a3,1c:28:af:b2:97:e0,...,9c:1c:12:1d:9e:40,9c:1c:12:1d:9e:41,9c:1c:12:1d:9e:42,9c:1c:12:1d:9e:43,9c:1c:12:1d:9e:45,bc:9f:e4:8b:52:a0,bc:9f:e4:8b:52:a2,bc:9f:e4:8b:52:a3,bc:9f:e4:8b:52:a4,bc:9f:e4:8b:52:a5
positionid,,,,,,,,,,,,,,,,,,,,,
1,1,1,5745,5745,100.0,100.0,100.0,100.0,100.0,-86.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-88.0,-74.0,-73.0,-73.0
2,1,1,5785,5785,100.0,100.0,-86.0,-86.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,-71.0,-66.0,-69.0,-68.0,-70.0
3,1,1,5785,5785,100.0,100.0,-84.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,-85.0,100.0
4,1,1,5785,5785,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,-75.0,100.0,100.0
5,1,1,5785,5785,100.0,100.0,100.0,100.0,100.0,-63.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,100.0,100.0,100.0
6,1,1,5785,5785,100.0,100.0,100.0,100.0,100.0,-63.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,100.0,100.0,100.0
7,1,1,5785,5785,100.0,100.0,100.0,100.0,100.0,-63.0,...,100.0,100.0,100.0,100.0,100.0,100.0,-86.0,100.0,100.0,100.0
8,1,1,5785,5785,100.0,100.0,100.0,100.0,100.0,-63.0,...,100.0,100.0,100.0,100.0,-88.0,100.0,100.0,100.0,100.0,100.0
9,1,1,5785,5785,-90.0,100.0,100.0,100.0,100.0,-63.0,...,-87.0,-86.0,-87.0,-88.0,100.0,100.0,100.0,100.0,100.0,100.0


In [27]:
majdataset = dataset.append(c['1'], ignore_index=True)
majdataset = majdataset.append(c['1'], ignore_index=True)
majdataset

,batimentid,floorid,centrefrequence0,frequency,00:4e:35:c8:ce:24,1c:28:af:b2:95:a0,1c:28:af:b2:95:a1,1c:28:af:b2:95:a2,1c:28:af:b2:95:a3,1c:28:af:b2:97:e0,...,9c:1c:12:1d:9e:45,bc:9f:e4:8b:52:a0,bc:9f:e4:8b:52:a2,bc:9f:e4:8b:52:a3,bc:9f:e4:8b:52:a4,bc:9f:e4:8b:52:a5,bssid,centerFreq0,level,ssid
0,1.0,1.0,5745.0,5745,100.0,100.0,100.0,100.0,100.0,-86.0,...,100.0,100.0,-88.0,-74.0,-73.0,-73.0,NaN,NaN,NaN,NaN
1,1.0,1.0,5785.0,5785,100.0,100.0,-86.0,-86.0,100.0,100.0,...,100.0,-71.0,-66.0,-69.0,-68.0,-70.0,NaN,NaN,NaN,NaN
2,1.0,1.0,5785.0,5785,100.0,100.0,-84.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,-85.0,100.0,NaN,NaN,NaN,NaN
3,1.0,1.0,5785.0,5785,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,-86.0,-75.0,100.0,100.0,NaN,NaN,NaN,NaN
4,1.0,1.0,5785.0,5785,100.0,100.0,100.0,100.0,100.0,-63.0,...,100.0,100.0,-86.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN
5,1.0,1.0,5785.0,5785,100.0,100.0,100.0,100.0,100.0,-63.0,...,100.0,100.0,-86.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN
6,1.0,1.0,5785.0,5785,100.0,100.0,100.0,100.0,100.0,-63.0,...,100.0,100.0,-86.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN
7,1.0,1.0,5785.0,5785,100.0,100.0,100.0,100.0,100.0,-63.0,...,-88.0,100.0,100.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN
8,1.0,1.0,5785.0,5785,-90.0,100.0,100.0,100.0,100.0,-63.0,...,100.0,100.0,100.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN
9,1.0,1.0,5785.0,5785,100.0,-84.0,-84.0,-84.0,-85.0,100.0,...,100.0,100.0,100.0,100.0,-83.0,100.0,NaN,NaN,NaN,NaN
